In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import skimage.transform
import skimage.filters
import skimage.feature
import skimage.restoration
from skimage.transform import resize
from skimage.filters import sobel
from skimage.feature import canny
from skimage.restoration import denoise_bilateral, denoise_tv_chambolle
from skimage.io import imread, imshow
from sklearn.cluster import KMeans
from scipy import misc
from skimage.color import rgb2gray
from skimage.io import imread, imshow
import shutil
import glob

import boto3
import sys
sys.path.insert(0,'/home/newton/images')
import os

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
# tf.enable_eager_execution()

import tensorflow_hub as hub
# import tensorflow_datasets as tfds

from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
base_dir= '/home/newton/images/gender_images'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
batch_size = 100
IMG_W = 178
IMG_H = 218

In [6]:
image_gen_train = ImageDataGenerator(rescale=1/255)
train_data_gen = image_gen_train.flow_from_directory(
                                                batch_size=batch_size, 
                                                directory=train_dir, 
                                                shuffle=True, 
                                                target_size=(IMG_W,IMG_H),
                                                class_mode='sparse'
                                                )

Found 8000 images belonging to 2 classes.


In [7]:
image_gen_val = ImageDataGenerator(rescale=1/255)

val_data_gen = image_gen_val.flow_from_directory(batch_size=batch_size, 
                                                 directory=val_dir, 
                                                 target_size=(IMG_W, IMG_H),
                                                 class_mode='sparse')

Found 2000 images belonging to 2 classes.


In [12]:
URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/2"

feature_extractor = hub.KerasLayer(URL)

feature_extractor.trainable = False

m1_gender = tf.keras.Sequential([
  feature_extractor,
  layers.Dense(5, activation='softmax')
])

In [29]:
temp_x = np.zeros((100, 224, 224, 3))
temp_y = np.zeros((100, ))
temp = [(temp_x, temp_y)]

In [27]:
train_data_gen[0][0].shape

(100, 178, 218, 3)

In [30]:
m1_gender.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [31]:
epochs = 15

history = m1_gender.fit(
    train_data_gen,
    epochs=epochs,
    validation_data=val_data_gen
)

ValueError: Please provide as model inputs either a single array or a list of arrays. You passed: x=[(array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        ...,

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
        

In [14]:
m1_gender.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.

In [ ]:
#train_path = '/home/newton/images/gender_train'
#dim = 218*178
#np2 = np.zeros(shape=(10000,dim))
#for i in range(10000):
#    filename = anno.iloc[i,0]    
#    img = rgb2gray(imread(os.path.join(train_path, filename)))
#    np1 = img.reshape(1,-1)
#    np2[i,:] = np1


In [ ]:
# write the train test data out.

In [ ]:
#np.savetxt('gender_train_test.txt', np2)

In [ ]:
#use the model in the lecture called fullpowermnist